<a href="https://colab.research.google.com/github/meti-94/TextGeneration/blob/main/personachat_beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title SSH Colab Dropbear
import apt, apt.debfile
import pathlib, stat, shutil, urllib.request, subprocess, getpass, time, tempfile
import secrets, json, re
import IPython.utils.io
import ipywidgets
import socket
import os
from IPython.display import clear_output

class _NoteProgress(apt.progress.base.InstallProgress, apt.progress.base.AcquireProgress, apt.progress.base.OpProgress):
  def __init__(self):
    apt.progress.base.InstallProgress.__init__(self)
    self._label = ipywidgets.Label()
    display(self._label)
    self._float_progress = ipywidgets.FloatProgress(min = 0.0, max = 1.0, layout = {'border':'1px solid #118800'})
    display(self._float_progress)

  def close(self):
    self._float_progress.close()
    self._label.close()

  def fetch(self, item):
    self._label.value = "fetch: " + item.shortdesc

  def pulse(self, owner):
    self._float_progress.value = self.current_items / self.total_items
    return True

  def status_change(self, pkg, percent, status):
    self._label.value = "%s: %s" % (pkg, status)
    self._float_progress.value = percent / 100.0

  def update(self, percent=None):
    self._float_progress.value = self.percent / 100.0
    self._label.value = self.op + ": " + self.subop

  def done(self, item=None):
    pass

class _MyApt:
  def __init__(self):
    self._progress = _NoteProgress()
    self._cache = apt.Cache(self._progress)

  def close(self):
    self._cache.close()
    self._cache = None
    self._progress.close()
    self._progress = None

  def update_upgrade(self):
    self._cache.update()
    self._cache.open(None)
    self._cache.upgrade()

  def commit(self):
    self._cache.commit(self._progress, self._progress)
    self._cache.clear()

  def installPkg(self, *args):
    for name in args:
      pkg = self._cache[name]
      if pkg.is_installed:
        print(f"{name} is already installed")
      else:
        pkg.mark_install()

  def installDebPackage(self, name):
    apt.debfile.DebPackage(name, self._cache).install()

  def deleteInstalledPkg(self, *args):
    for pkg in self._cache:
      if pkg.is_installed:
        for name in args:
          if pkg.name.startswith(name):
            #print(f"Delete {pkg.name}")
            pkg.mark_delete()

def _download(url, path):
  try:
    with urllib.request.urlopen(url) as response:
      with open(path, 'wb') as outfile:
        shutil.copyfileobj(response, outfile)
  except:
    print("Failed to download ", url)
    raise

  return True

def _setupssh():
  #Inpur Username
  print("Create a username :")
  user_name = input()
  subprocess.run(["useradd", "-s", "/bin/bash", "-m", user_name])
  subprocess.run(["adduser", user_name, "sudo"], check = True)
  clear_output()

  #Inpur Password
  print("Create a password :")
  user_password = getpass.getpass()
  subprocess.run(["chpasswd"], input = f"{user_name}:{user_password}", universal_newlines = True)
  clear_output()

  #Set your ngrok Authtoken.
  print("Copy & paste your tunnel authtoken from https://dashboard.ngrok.com/auth")
  print("(You need to sign up for ngrok and login.)")
  ngrok_token = getpass.getpass()
  clear_output()

  #Set your ngrok region.
  print("Select your ngrok region :")
  print("us - United States (Ohio)")
  print("eu - Europe (Frankfurt)")
  print("ap - Asia/Pacific (Singapore)")
  print("au - Australia (Sydney)")
  print("sa - South America (Sao Paulo)")
  print("jp - Japan (Tokyo)")
  print("in - India (Mumbai)")
  ngrok_region = input()
  clear_output()

  #SSH Dropbear
  my_apt = _MyApt()
  my_apt.installPkg("dropbear")
  my_apt.commit()
  my_apt.close()

  #Set Dropbear
  f = open("../etc/default/dropbear", "w")
  f.write("""# the TCP port that Dropbear listens on
DROPBEAR_PORT=443

# any additional arguments for Dropbear
DROPBEAR_EXTRA_ARGS=

# specify an optional banner file containing a message to be
# sent to clients before they connect, such as "/etc/issue.net"
DROPBEAR_BANNER=""

# RSA hostkey file (default: /etc/dropbear/dropbear_rsa_host_key)
#DROPBEAR_RSAKEY="/etc/dropbear/dropbear_rsa_host_key"

# DSS hostkey file (default: /etc/dropbear/dropbear_dss_host_key)
#DROPBEAR_DSSKEY="/etc/dropbear/dropbear_dss_host_key"

# ECDSA hostkey file (default: /etc/dropbear/dropbear_ecdsa_host_key)
#DROPBEAR_ECDSAKEY="/etc/dropbear/dropbear_ecdsa_host_key"

# Receive window size - this is a tradeoff between memory and
# network performance
DROPBEAR_RECEIVE_WINDOW=65536
""")
  f.close()

  #SSH Service Restart
  subprocess.run(["service", "dropbear", "restart"])

  #Root Password
  root_password = user_password
  subprocess.run(["chpasswd"], input = f"root:{root_password}", universal_newlines = True)

  # Ngrok Tunnel
  if not os.path.exists('ngrok.zip'):
    _download("https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip", "ngrok.zip")
    shutil.unpack_archive("ngrok.zip")
    pathlib.Path("ngrok").chmod(stat.S_IXUSR)

  subprocess.run(["./ngrok", "authtoken", ngrok_token])
  ngrok_proc = subprocess.Popen(["./ngrok", "tcp", "-region", ngrok_region, "443"])
  time.sleep(2)

  with urllib.request.urlopen("http://localhost:4040/api/tunnels") as response:
    url = json.load(response)['tunnels'][0]['public_url']
    m = re.match("tcp://(.+):(\d+)", url)

  hostname = m.group(1)
  port = m.group(2)
  
  msg = ""
  msg += "="*48 + "\n"
  msg += "Command to connect to the ssh server:\n"
  msg += f"ssh -p {port} {user_name}@{socket.gethostbyname(hostname)}\n"
  msg += "="*48 + "\n"
  msg += "SSH Login NetMod and Others:\n"
  msg += f"Host            : {socket.gethostbyname(hostname)}\n"
  msg += f"Port (Dropbear) : {port}\n"
  msg += f"Username        : {user_name}\n"
  msg += f"Password        : {user_password}\n"
  msg += "="*48 + "\n"
  return msg

def startColab():
  msg = _setupssh()
  print(msg)

startColab()

In [ ]:
!wget http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
!tar -xf  simple-examples.tgz
!mkdir data
!mv /content/simple-examples/data/ptb.train.txt data/
!mv /content/simple-examples/data/ptb.valid.txt data/
!mv /content/simple-examples/data/ptb.test.txt data/
!rm -rf ./simple_examples

In [1]:
!pip install "ray[tune]" transformers datasets==1.1.2
# !pip install transformers
!ray install-nightly
!pip install rouge_score
!pip install -U ray
# !pip install -U transformers datasets
!pip install tensorboardX


     |████████████████████████████████| 47.5MB 63kB/s 
     |████████████████████████████████| 2.1MB 38.0MB/s 
     |████████████████████████████████| 153kB 36.3MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 133kB 44.5MB/s 
     |████████████████████████████████| 204kB 39.2MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
     |████████████████████████████████| 81kB 8.1MB/s 
     |████████████████████████████████| 1.3MB 33.3MB/s 
     |████████████████████████████████| 3.1MB 37.7MB/s 
     |████████████████████████████████| 122kB 42.5MB/s 
     |████████████████████████████████| 3.3MB 36.4MB/s 
     |████████████████████████████████| 901kB 35.9MB/s 
     |████████████████████████████████| 245kB 41.5MB/s 
     |████████████████████████████████| 92kB 8.5MB/s 
     |████████████████████████████████| 296kB 39.4MB/s 
     |████████████████████████████████| 143kB 23.6MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp37-

In [2]:
import datasets
datasets.__version__

'1.1.2'

In [8]:
!wget https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json
!mkdir models

--2021-04-21 13:34:49--  https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.136.254
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.136.254|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209850483 (200M) [application/json]
Saving to: ‘personachat_self_original.json’

personachat_self_or 100%[===================>] 200.13M  35.5MB/s    in 6.3s    

2021-04-21 13:34:56 (31.8 MB/s) - ‘personachat_self_original.json’ saved [209850483/209850483]



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import ray
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import AsyncHyperBandScheduler

from torch.utils.data import Dataset, DataLoader
from transformers import EncoderDecoderModel, BertTokenizer, BertModel, BertConfig
import torch
from transformers import AdamW
from tqdm import tqdm
import json
import random 
from sklearn.model_selection import train_test_split
from datetime import datetime
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from tqdm import tqdm_notebook as tqdm
import logging
import datasets
logging.basicConfig(level=logging.DEBUG)

In [5]:
SPECIAL_TOKENS = ["<bos>", "<eos>", "<persona>", "<speaker1>", "<speaker2>", "<pad>"]

ATTR_TO_SPECIAL_TOKEN = {'bos_token': '<bos>', 'eos_token': '<eos>', 'pad_token': '<pad>',
                         'additional_special_tokens': ['<speaker1>', '<speaker2>', '<persona>']}

encoder_max_length=512
decoder_max_length=128

def read_data(data_json_path='/content/personachat_self_original.json'):
  with open(data_json_path) as json_file:
    data_dict = json.load(json_file)
  return data_dict

def data_to_samples(data_dict, test=False):
  samples=[]
  for dialogue in (data_dict['train'] if test==False else data_dict['valid']):
    original_persona = dialogue['personality']
    
    for item in dialogue['utterances']:
      original_persona = [original_persona[-1]] + original_persona[:-1]
      history = item['history']
      response = item['candidates'][-1]
      samples.append({
          'persona':original_persona,
          'history':history,
          'response':response
      })
  return samples
  
def bertified(samples):
  bertified_data = []
  for item in samples:
    persona = ' <persona> '.join(item['persona'])
    persona = '<bos> ' + persona
    history = ''
    speakers = [" <speaker1> ", " <speaker2> "]
    speaker = 0
    for hst in item['history'][::-1]:
      history = speakers[speaker] + hst + history
      speaker = 1 - speaker
    response = '<speaker2> ' + item['response'] + ' <eos>'
    bertified_data.append({
          'persona':persona.replace('  ', ' '),
          'history':history.replace('  ', ' '),
          'input': persona.replace('  ', ' ')+' '+history.replace('  ', ' '), 
          'response':response.replace('  ', ' ')
      })
  return bertified_data

def bertified_to_model_food(bertified_data, tokenizer):
    model_food = []
    pbar = tqdm(bertified_data)
    pbar.set_description('Tokenizing Data Sample ...')
    for item in pbar:
        inputs = tokenizer(item['input'], add_special_tokens=True, max_length=encoder_max_length, truncation=True, padding="max_length")
        outputs = tokenizer(item['response'], add_special_tokens=True, max_length=decoder_max_length, truncation=True, padding="max_length")
        _item = {}
        _item["input_ids"] = inputs.input_ids
        _item["attention_mask"] = inputs.attention_mask
        _item["decoder_input_ids"] = outputs.input_ids
        _item["decoder_attention_mask"] = outputs.attention_mask
        _item["labels"] = outputs.input_ids.copy()
        model_food.append(_item)
    return model_food 

In [6]:
class PersonaDataset_v3(Dataset):
  '''
      Convert Data to proper Tensor dataset
  '''
  def __init__(self, samples):
    self.samples = samples
    self.n_samples = len(self.samples)

  def __getitem__(self, index):
    # returns specific item
    return self.samples[index]

  def __len__(self):
    return self.n_samples

In [9]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained("prajjwal1/bert-tiny", "prajjwal1/bert-tiny")
tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-tiny")
tokenizer.add_special_tokens(ATTR_TO_SPECIAL_TOKEN)
model.get_encoder().resize_token_embeddings(len(tokenizer))
model.get_decoder().resize_token_embeddings(len(tokenizer))

data = read_data()
data_samples = data_to_samples(data)
bertified_data = bertified(data_samples)
train, valid = train_test_split(bertified_data[:100], test_size=0.15, random_state=99)
train = bertified_to_model_food(train, tokenizer)
valid = bertified_to_model_food(valid, tokenizer)
test_data = read_data()
test_data_samples = data_to_samples(test_data, True)
test = bertified(test_data_samples)
test = bertified_to_model_food(test[:100], tokenizer)



DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationsh

In [10]:
# train_dataset = PersonaDataset_v3(list(zip(*map(one_line_tokenizer, tqdm(train)))))
train_dataset = PersonaDataset_v3(train[:100])
valid_dataset = PersonaDataset_v3(valid[:100])
test_dataset = PersonaDataset_v3(test[:100])



In [11]:
rouge = datasets.load_metric("rouge")
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): s3.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/metrics/rouge/rouge.py HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com:443
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "HEAD /huggingface/datasets/1.1.2/metrics/rouge/rouge.py HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 140211271044816 on /root/.cache/huggingface/datasets/336a0cbdd138e0d6e1361544ed2c56565192be45b5398cd55ac988d4b3fd1455.a89f1fa0750909f2d149b1ecabc808fb66cb865c94bb8bbb135c55deb50da2d7.py.lock
INFO:filelock:Lock 140211271044816 acquired on /root/.cache/huggingface/datasets/336a0cbdd138e0d6e1361544ed2c56565192be45b5398cd55ac988d4b3fd1455.a89f1fa0750909f2d149b1ecabc808fb66cb865c94bb8bbb135c55deb50da2d7.py.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com:44

DEBUG:filelock:Attempting to release lock 140211271044816 on /root/.cache/huggingface/datasets/336a0cbdd138e0d6e1361544ed2c56565192be45b5398cd55ac988d4b3fd1455.a89f1fa0750909f2d149b1ecabc808fb66cb865c94bb8bbb135c55deb50da2d7.py.lock
INFO:filelock:Lock 140211271044816 released on /root/.cache/huggingface/datasets/336a0cbdd138e0d6e1361544ed2c56565192be45b5398cd55ac988d4b3fd1455.a89f1fa0750909f2d149b1ecabc808fb66cb865c94bb8bbb135c55deb50da2d7.py.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com:443


DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "HEAD /huggingface/datasets/1.1.2/metrics/rouge/dataset_infos.json HTTP/1.1" 404 0
DEBUG:filelock:Attempting to acquire lock 140209947123664 on /root/.cache/huggingface/datasets/336a0cbdd138e0d6e1361544ed2c56565192be45b5398cd55ac988d4b3fd1455.a89f1fa0750909f2d149b1ecabc808fb66cb865c94bb8bbb135c55deb50da2d7.py.lock
INFO:filelock:Lock 140209947123664 acquired on /root/.cache/huggingface/datasets/336a0cbdd138e0d6e1361544ed2c56565192be45b5398cd55ac988d4b3fd1455.a89f1fa0750909f2d149b1ecabc808fb66cb865c94bb8bbb135c55deb50da2d7.py.lock
DEBUG:filelock:Attempting to release lock 140209947123664 on /root/.cache/huggingface/datasets/336a0cbdd138e0d6e1361544ed2c56565192be45b5398cd55ac988d4b3fd1455.a89f1fa0750909f2d149b1ecabc808fb66cb865c94bb8bbb135c55deb50da2d7.py.lock
INFO:filelock:Lock 140209947123664 released on /root/.cache/huggingface/datasets/336a0cbdd138e0d6e1361544ed2c56565192be45b5398cd55ac988d4b3fd1455.a89f1fa0750909f2d14

In [12]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size

# model.config.max_length = 142
# model.config.min_length = 56
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [13]:
batch_size = 128
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/models/",
    do_eval=True,
    # do_train = True,
    # do_train = True,
    evaluation_strategy="steps",
    
    eval_steps=2,
    logging_steps=2,


    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    predict_with_generate=True,
    
    
    fp16=True, 
    
    save_strategy = 'steps',
    save_steps=35,
    save_total_limit = 2
    
    # logging_steps=1000,
    # save_steps=500,
    # eval_steps=7500,
    # warmup_steps=2000,
    # save_total_limit=3,
)
# instantiate trainer
def model_init():
    return model
trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)



# trainer.train()

In [14]:
# from ray import tune
# from ray.tune.suggest.hyperopt import HyperOptSearch
# from ray.tune.schedulers import AsyncHyperBandScheduler

trainer.hyperparameter_search(
    direction="maximize", 
    backend="ray", 
    n_trials=2, # number of trials
    # n_jobs=2  # number of parallel jobs, if multiple GPUs
)

INFO:ray.tune.tune:Initializing Ray automatically.For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run`.
2021-04-21 13:36:38,949	INFO services.py:1264 -- View the Ray dashboard at http://127.0.0.1:8265


== Status ==
Memory usage on this node: 3.2/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/7.35 GiB heap, 0.0/3.68 GiB objects (0.0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/_objective_2021-04-21_13-36-41
Number of trials: 2/2 (1 PENDING, 1 RUNNING)
+------------------------+----------+-------+-----------------+--------------------+-------------------------------+---------+
| Trial name             | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |    seed |
|------------------------+----------+-------+-----------------+--------------------+-------------------------------+---------|
| _objective_9b23c_00000 | RUNNING  |       |     5.61152e-06 |                  5 |                            64 | 8.15396 |
| _objective_9b23c_00001 | PENDING  |       |     1.56207e-05 |                  2 |                            16 | 7.08379 |
+------------------------+----------+-------+---------

(pid=494) 2021-04-21 13:36:43.401103: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


== Status ==
Memory usage on this node: 4.1/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/7.35 GiB heap, 0.0/3.68 GiB objects (0.0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/_objective_2021-04-21_13-36-41
Number of trials: 2/2 (1 PENDING, 1 RUNNING)
+------------------------+----------+-------+-----------------+--------------------+-------------------------------+---------+
| Trial name             | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |    seed |
|------------------------+----------+-------+-----------------+--------------------+-------------------------------+---------|
| _objective_9b23c_00000 | RUNNING  |       |     5.61152e-06 |                  5 |                            64 | 8.15396 |
| _objective_9b23c_00001 | PENDING  |       |     1.56207e-05 |                  2 |                            16 | 7.08379 |
+------------------------+----------+-------+---------

(pid=494) /usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
(pid=494)   "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


...

(pid=493) <IPython.core.display.HTML object>
(pid=493) <IPython.core.display.HTML object>
(pid=493) <IPython.core.display.HTML object>
Result for _objective_9b23c_00001:
  date: 2021-04-21_13-37-11
  done: false
  eval_loss: 10.923986434936523
  eval_rouge2_fmeasure: 0.0
  eval_rouge2_precision: 0.0
  eval_rouge2_recall: 0.0
  eval_runtime: 1.5591
  eval_samples_per_second: 9.621
  experiment_id: 39a3a95d87ac474d8c529db81a36dd12
  hostname: 9a16a58603ea
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  objective: 0.0
  pid: 493
  time_since_restore: 8.163029670715332
  time_this_iter_s: 8.163029670715332
  time_total_s: 8.163029670715332
  timestamp: 1619012231
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 9b23c_00001
  
== Status ==
Memory usage on this node: 4.2/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/7.35 GiB heap, 0.0/3.68 GiB objects (0.0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/_objective_

2021-04-21 13:37:22,638	INFO tune.py:549 -- Total run time: 41.15 seconds (40.76 seconds for the tuning loop).


Result for _objective_9b23c_00001:
  date: 2021-04-21_13-37-22
  done: true
  eval_loss: 10.064268112182617
  eval_rouge2_fmeasure: 0.0
  eval_rouge2_precision: 0.0
  eval_rouge2_recall: 0.0
  eval_runtime: 1.5345
  eval_samples_per_second: 9.775
  experiment_id: 39a3a95d87ac474d8c529db81a36dd12
  experiment_tag: 1_learning_rate=1.5621e-05,num_train_epochs=2,per_device_train_batch_size=16,seed=7.0838
  hostname: 9a16a58603ea
  iterations_since_restore: 6
  node_ip: 172.28.0.2
  objective: 0.0
  pid: 493
  time_since_restore: 18.793177843093872
  time_this_iter_s: 2.0758109092712402
  time_total_s: 18.793177843093872
  timestamp: 1619012242
  timesteps_since_restore: 0
  training_iteration: 6
  trial_id: 9b23c_00001
  
== Status ==
Memory usage on this node: 4.2/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.35 GiB heap, 0.0/3.68 GiB objects (0.0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/_objective_2021-04-21_13-36-41
Number of 

(pid=493) Error in atexit._run_exitfuncs:
(pid=493) Traceback (most recent call last):
(pid=493)   File "/usr/local/lib/python3.7/dist-packages/IPython/core/history.py", line 786, in writeout_cache
(pid=493)     self._writeout_input_cache(conn)
(pid=493)   File "/usr/local/lib/python3.7/dist-packages/IPython/core/history.py", line 770, in _writeout_input_cache
(pid=493)     (self.session_number,)+line)
(pid=493) sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 139631387977472 and this is thread id 139635791624064.


BestRun(run_id='9b23c_00000', objective=0.0, hyperparameters={'learning_rate': 5.61151641533451e-06, 'num_train_epochs': 5, 'seed': 8.153956804780389, 'per_device_train_batch_size': 64})

In [13]:
!rm -rf /root/ray_results/